### 타이타닉 따라해보기 1

### 1.라이브러리 불러오기

In [1]:

import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') #matplotlib 스타일 관련
%matplotlib inline

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC #서포트 벡터머신
from sklearn.ensemble import RandomForestClassifier # 랜덤포레스트
from sklearn.neighbors import KNeighborsClassifier #K- 최근접이웃


### 2. 데이터읽기

In [4]:
train_df = pd.read_csv('C:/Users/Admin/Desktop/success/Python/taitanic/train.csv')
test_df = pd.read_csv('C:/Users/Admin/Desktop/success/Python/taitanic/test.csv')

# train_df.head()
test_df.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
#데이터의 정보 확인하기 -> info 메서드
train_df.info()
print('-'*20)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
--------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    

In [6]:
train_df = train_df.drop(['PassengerId','Name','Ticket'],axis = 1)
test_df = test_df.drop(['Name','Ticket'], axis = 1)

### 3.데이터 하나하나 처리하기
#### 3.1 Pclass

In [7]:
### 3.1 Pcalss
train_df['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [8]:
### 'Pclass'는 범주형 데이터이므로 원-핫-인코딩을 pd.get_dummies()매서드로 인코딩
##pd.get_dummies -> 결측값을 제외하고 0과1로 구성된 더미값을 만들어준다.
##이거 드롭을 두번이상 실행하면 오류남 ㅇㅇ
pclass_train_dummies = pd.get_dummies(train_df['Pclass'])
pclass_test_dummies = pd.get_dummies(test_df['Pclass'])


train_df.drop(['Pclass'], axis=1, inplace=True)
test_df.drop(['Pclass'], axis=1, inplace=True)

train_df =train_df.join(pclass_train_dummies)
test_df = test_df.join(pclass_test_dummies)




In [9]:
train_df.head()

,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,1,2,3
0,0,male,22.0,1,0,7.2500,NaN,S,0,0,1
1,1,female,38.0,1,0,71.2833,C85,C,1,0,0
2,1,female,26.0,0,0,7.9250,NaN,S,0,0,1
3,1,female,35.0,1,0,53.1000,C123,S,1,0,0
4,0,male,35.0,0,0,8.0500,NaN,S,0,0,1


#### 3.2 Sex
####  성별 또한 남과 여로 나뉘므로 원핫인코딩 진행한다.

In [10]:
sex_train_dummies = pd.get_dummies(train_df['Sex'])
sex_test_dummies = pd.get_dummies(test_df['Sex'])

sex_train_dummies.columns = ['Female', 'Male']
sex_test_dummies.columns = ['Female', 'Male']

train_df.drop(['Sex'], axis=1, inplace = True)
test_df.drop(['Sex'], axis=1, inplace = True)

train_df = train_df.join(sex_train_dummies)
test_df = test_df.join(sex_test_dummies)

In [11]:
train_df

,Survived,Age,SibSp,Parch,Fare,Cabin,Embarked,1,2,3,Female,Male
0,0,22.0,1,0,7.2500,NaN,S,0,0,1,0,1
1,1,38.0,1,0,71.2833,C85,C,1,0,0,1,0
2,1,26.0,0,0,7.9250,NaN,S,0,0,1,1,0
3,1,35.0,1,0,53.1000,C123,S,1,0,0,1,0
4,0,35.0,0,0,8.0500,NaN,S,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.0,0,0,13.0000,NaN,S,0,1,0,0,1
887,1,19.0,0,0,30.0000,B42,S,1,0,0,1,0
888,0,NaN,1,2,23.4500,NaN,S,0,0,1,1,0
889,1,26.0,0,0,30.0000,C148,C,1,0,0,0,1


#### 3.3Age
#### 나이는 연속형 데이터이므로, 큰처리는 필요없음(but 카테고리화 하여 일부 알고리즘에 더 유용한 결과를 만들 수 있음)
#### 어쨌거나 일부 NaN 데이터가 있으니 이를 1랜덤/2평균값/3중앙값/4데이터버리기 중 2평균값으로 넣어줄 것임

In [12]:
train_df['Age'].fillna(train_df['Age'].mean(), inplace=True)
test_df['Age'].fillna(train_df['Age'].mean(), inplace=True)

#### 3.4SibSp & Panch : 형제 자매와 부모님은 가족으로 함께 처리0, 바꿀필요X

#### 3.5 Fare :탑승료, test데이터셋에 1개의 데이터가 비어있음. 아마 디카프리오인듯 :) 우선 빈 부분을 fillna매서드로 채우자!
#### 데이터 누락이 아니 무단 탑승으로 생각해 0으로 입력

In [13]:
test_df["Fare"].fillna(0, inplace=True)

#### 3.6 Cabin: 객실, NaN인 데이터가 너무 많아서 버린다.

In [14]:
train_df = train_df.drop(['Cabin'], axis = 1)
test_df = test_df.drop(['Cabin'], axis = 1)

#### 3.7 Embarked: 탑승항구

In [17]:
train_df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
test_df['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

#### 위의 결과를 보았을 떄, S가 대다수이고 일부데이터가 비어있는 것을 알 수 있음, 빈부분을 모두 S로 채우고 시작

In [23]:
train_df['Embarked'].fillna('S', inplace=True)
test_df['Embarked'].fillna('S', inplace = True)

In [24]:
embarked_train_dummies = pd.get_dummies(train_df['Embarked'])
embarked_test_dummies = pd.get_dummies(test_df['Embarked'])

embarked_train_dummies.columns = ['S', 'C', 'Q']
embarked_test_dummies.columns = ['S', 'C', 'Q']

train_df.drop(['Embarked'], axis = 1, inplace=True)
test_df.drop(['Embarked'], axis = 1, inplace=True)

train_df = train_df.join(embarked_train_dummies)
test_df = test_df.join(embarked_test_dummies)

In [ ]:
test_df

### 4.데이터 나누기
#### 학습용 데이터를 위해 데이터를 나누어야 한다

In [28]:
#(정보,생존 여부)와 같은 형태를 위해 다음과 같이 데이터를 나눈다
X_train = train_df.drop('Survived', axis=1)
Y_train = train_df['Survived']
X_test = test_df.drop("PassengerId",  axis = 1).copy()

### 5.머신러닝 알고리즘 적용하기

#### 5.1 Losgistic Regression

In [33]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_test)

logreg.score(X_train, Y_train)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8058361391694725

In [34]:
#### 5.2Support Vector Machines
svc = SVC()

svc.fit(X_train,Y_train)

Y_pred = svc.predict(X_test)

svc.score(X_train, Y_train)

### 여기서 궁금증 하나?  나는 왜 결과가 블로그랑은 다르게 나왔을까
### 0.6이 내가 해서 나온값. 블로그값 ->0.88

0.6868686868686869

#### 5.3 Random Forests

In [36]:
random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

0.9820426487093153

#### 랜덤 포레스트가 가장 좋은 결과를 내는 것을 알 수 있다